In [1]:
# https://keras.io/
!pip install -q keras
import keras

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
tensorflow-gpu 1.9.0 has requirement tensorboard<1.10.0,>=1.9.0, but you'll have tensorboard 1.10.0 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.optimizers import SGD

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 250
l = 5
num_filter = 12
compression = 0.5
dropout_rate = 0.20

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Rotate images by 45 degrees
datagen1 = ImageDataGenerator(rotation_range=45)

# fit parameters from data
datagen1.fit(x_train)

# Get 1000 augmented images and append to the training data
for x_rotated_images_batch, y_rotated_images_batch in datagen1.flow(x_train, y_train, batch_size=1000):
    x_train = np.append(x_train, x_rotated_images_batch, axis=0)
    y_train = np.append(y_train, y_rotated_images_batch, axis=0)
    break

# Horizontally flip image
datagen2 = ImageDataGenerator(horizontal_flip=True)

# fit parameters from data
datagen2.fit(x_train)  

# Get 1000 augmented horizontally flipped images and append to the training data
for x_flipped_images_batch, y_flipped_images_batch in datagen2.flow(x_train, y_train, batch_size=1000):
    x_train = np.append(x_train, x_flipped_images_batch, axis=0)
    y_train = np.append(y_train, y_flipped_images_batch, axis=0)
    break


In [7]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_1_1 = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3),  use_bias=False ,padding='same')(relu)
        Conv2D_5_5 = Conv2D(int(num_filter*compression), (5,5),  use_bias=False ,padding='same')(relu)
        Conv2D_7_7 = Conv2D(int(num_filter*compression), (7,7),  use_bias=False ,padding='same')(relu)
        Conv2D_all_filters = Concatenate(axis=-1)([Conv2D_1_1,Conv2D_3_3,Conv2D_5_5,Conv2D_7_7])
        if dropout_rate>0:
          Conv2D_all_filters = Dropout(dropout_rate)(Conv2D_all_filters)
        concat = Concatenate(axis=-1)([temp,Conv2D_all_filters])
        
        temp = concat
        
    return temp

In [8]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return AvgPooling

In [9]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [10]:
input = Input(shape=(img_height, img_width, channel,))
layer1_transition = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

layer2_block = add_denseblock(layer1_transition, num_filter, dropout_rate)
layer2_transition = add_transition(layer2_block, num_filter, dropout_rate)
skip_connection1 = layer2_transition

layer3_block = add_denseblock(layer2_transition, num_filter, dropout_rate)
layer3_transition = add_transition(layer3_block, num_filter, dropout_rate)

layer4_block = add_denseblock(layer3_transition, num_filter, dropout_rate)
layer4_transition = add_transition(layer4_block, num_filter, dropout_rate)

layer5_skip_connection_block = add_denseblock(skip_connection1, num_filter, dropout_rate)
layer5_skip_connection_block = Conv2D(int(num_filter*compression), (5,5), use_bias=False)(layer5_skip_connection_block)
layer5_skip_connection_block = Conv2D(int(num_filter*compression), (5,5), use_bias=False)(layer5_skip_connection_block)
layer5_skip_connection_block = Conv2D(int(num_filter*compression), (5,5), use_bias=False)(layer5_skip_connection_block)
layer5_block = Concatenate(axis=-1)([layer5_skip_connection_block, layer4_transition])
layer5_transition = add_transition(layer5_block, num_filter, dropout_rate)

layer6_block = add_denseblock(layer5_transition,  num_filter, dropout_rate)
output = output_layer(layer6_block)

In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

## Callbacks For Early Stopping and Saving the Best Model

In [13]:
from keras.callbacks import Callback
class EarlyStoppingByValidationAccuracy(Callback):
    def __init__(self, monitor='val_acc', value=1.0, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        accuracy_value= logs.get(self.monitor)
       
        if accuracy_value >= self.value:
            self.model.stop_training = True
            if self.verbose == 1:
                print("Epoch %d: Threshold for early stopping has reached" % (epoch + 1))   

In [ ]:
from keras.callbacks import ModelCheckpoint
callbacks_list=[]
model_save_path= "best_model-CIFAR10-monimoy-my_computer_latest2.h5"
callbacks_list.append(EarlyStoppingByValidationAccuracy(monitor='val_acc',  value=0.9201, verbose=1))
callbacks_list.append(ModelCheckpoint(model_save_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 52000 samples, validate on 10000 samples
Epoch 1/250
52000/52000 [==============================] - 189s 4ms/step - loss: 1.9231 - acc: 0.2633 - val_loss: 1.7646 - val_acc: 0.3311

Epoch 00001: val_acc improved from -inf to 0.33110, saving model to best_model-CIFAR10-monimoy-my_computer_latest2.h5
Epoch 2/250
52000/52000 [==============================] - 171s 3ms/step - loss: 1.6160 - acc: 0.3847 - val_loss: 1.5035 - val_acc: 0.4367

Epoch 00002: val_acc improved from 0.33110 to 0.43670, saving model to best_model-CIFAR10-monimoy-my_computer_latest2.h5
Epoch 3/250
52000/52000 [==============================] - 170s 3ms/step - loss: 1.4753 - acc: 0.4528 - val_loss: 1.5222 - val_acc: 0.4533

Epoch 00003: val_acc improved from 0.43670 to 0.45330, saving model to best_model-CIFAR10-monimoy-my_computer_latest2.h5
Epoch 4/250
52000/52000 [==============================] - 171s 3ms/step - loss: 1.3677 - acc: 0.4972 - val_loss: 1.4259 - val_acc: 0.5003

Epoch 00004: val_acc improved 

Epoch 38/250
52000/52000 [==============================] - 173s 3ms/step - loss: 0.5739 - acc: 0.8016 - val_loss: 0.7409 - val_acc: 0.7690

Epoch 00038: val_acc did not improve from 0.77300
Epoch 39/250
52000/52000 [==============================] - 173s 3ms/step - loss: 0.5670 - acc: 0.8045 - val_loss: 0.8662 - val_acc: 0.7334

Epoch 00039: val_acc did not improve from 0.77300
Epoch 40/250
52000/52000 [==============================] - 173s 3ms/step - loss: 0.5602 - acc: 0.8067 - val_loss: 0.7324 - val_acc: 0.7673

Epoch 00040: val_acc did not improve from 0.77300
Epoch 41/250
52000/52000 [==============================] - 171s 3ms/step - loss: 0.5556 - acc: 0.8087 - val_loss: 0.7711 - val_acc: 0.7590

Epoch 00041: val_acc did not improve from 0.77300
Epoch 42/250
52000/52000 [==============================] - 172s 3ms/step - loss: 0.5483 - acc: 0.8126 - val_loss: 0.7930 - val_acc: 0.7620

Epoch 00042: val_acc did not improve from 0.77300
Epoch 43/250
52000/52000 [===================

52000/52000 [==============================] - 173s 3ms/step - loss: 0.3826 - acc: 0.8673 - val_loss: 0.6002 - val_acc: 0.8234

Epoch 00079: val_acc improved from 0.80670 to 0.82340, saving model to best_model-CIFAR10-monimoy-my_computer_latest2.h5
Epoch 80/250
52000/52000 [==============================] - 171s 3ms/step - loss: 0.3835 - acc: 0.8689 - val_loss: 0.7053 - val_acc: 0.7969

Epoch 00080: val_acc did not improve from 0.82340
Epoch 81/250
52000/52000 [==============================] - 172s 3ms/step - loss: 0.3784 - acc: 0.8699 - val_loss: 0.7243 - val_acc: 0.7952

Epoch 00081: val_acc did not improve from 0.82340
Epoch 82/250
52000/52000 [==============================] - 172s 3ms/step - loss: 0.3746 - acc: 0.8716 - val_loss: 0.7744 - val_acc: 0.7882

Epoch 00082: val_acc did not improve from 0.82340
Epoch 83/250
52000/52000 [==============================] - 172s 3ms/step - loss: 0.3705 - acc: 0.8727 - val_loss: 0.8590 - val_acc: 0.7602

Epoch 00083: val_acc did not improve 

52000/52000 [==============================] - 173s 3ms/step - loss: 0.2286 - acc: 0.9202 - val_loss: 0.7493 - val_acc: 0.8138

Epoch 00162: val_acc did not improve from 0.84030
Epoch 163/250
52000/52000 [==============================] - 173s 3ms/step - loss: 0.2269 - acc: 0.9208 - val_loss: 0.7402 - val_acc: 0.8195

Epoch 00163: val_acc did not improve from 0.84030
Epoch 164/250
52000/52000 [==============================] - 173s 3ms/step - loss: 0.2246 - acc: 0.9220 - val_loss: 0.7628 - val_acc: 0.8152

Epoch 00164: val_acc did not improve from 0.84030
Epoch 165/250
52000/52000 [==============================] - 173s 3ms/step - loss: 0.2262 - acc: 0.9206 - val_loss: 0.7819 - val_acc: 0.8203

Epoch 00165: val_acc did not improve from 0.84030
Epoch 166/250
52000/52000 [==============================] - 172s 3ms/step - loss: 0.2224 - acc: 0.9233 - val_loss: 0.7212 - val_acc: 0.8219

Epoch 00166: val_acc did not improve from 0.84030
Epoch 167/250
52000/52000 [===========================

In [ ]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")